**1. Download Dataset using Pandas**


*Using only Pandas library to download from wiki.*


*Datadrame created as per requirement "The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood"*


In [1]:
import pandas as pd
import numpy as np

link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

tables = pd.read_html(link)
src_dtframe = pd.DataFrame(tables[0])
stg_dtframe = src_dtframe.rename(columns = { 'Postcode' : 'PostalCode'})
stg_dtframe.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [2]:
#Dataframe processed to consider the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
stg_dtframe = stg_dtframe[stg_dtframe['Borough'] != 'Not assigned']
stg_dtframe.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table
stg_dtframe1 = stg_dtframe
gp =stg_dtframe1.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join) 
stg_dtframe2 = pd.DataFrame(gp)
stg_dtframe2.reset_index(inplace = True)
stg_dtframe2

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [4]:
#If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough
stg_dtframe3 = stg_dtframe2

stg_dtframe3.loc[stg_dtframe3['Neighbourhood'] == 'Not assigned','Neighbourhood'] = stg_dtframe3['Borough']
stg_dtframe3[stg_dtframe3['Postal Code'] == 'M7A']

,Postal Code,Borough,Neighbourhood
85,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
#use the .shape method to print the number of rows of our dataframe

stg_dtframe3.shape

(103, 3)

**Used Geospatial Data to generate dataframe and joined it with postal dataframe to create a combined frame**

In [6]:
!wget -q -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data
df_geospatial = pd.read_csv('Geospatial_data.csv')

In [7]:
df_geospatial.rename(columns ={'Postal Code':'PostalCode'},inplace=True)
df_geospatial.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df_geospatial.columns

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')

In [9]:
merged_postals = pd.merge(left=stg_dtframe3,right=df_geospatial, left_on='Postal Code', right_on='PostalCode')
merged_postals.head()

,Postal Code,Borough,Neighbourhood,PostalCode,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


**Installed geopy and folium package**

In [10]:
!pip install folium

In [11]:
!pip install geopy

In [12]:
import folium
from geopy.geocoders import Nominatim

**Selected Borough that contains Toronto (as per requirement)**

In [13]:
map_toronto = folium.Map(width=500,height=500,location=[43.6532, -79.3832], zoom_start=11
                         
                        )
toronto_postals = merged_postals[merged_postals['Borough'].str.contains('Toronto')]
toronto_postals_copy = toronto_postals
toronto_postals.head()

,Postal Code,Borough,Neighbourhood,PostalCode,Latitude,Longitude
37,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",M4L,43.668999,-79.315572
43,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790


**Visualising Toronto Neighborhood**

In [14]:
# add markers to map
for lat, lng, Borough, Neighbourhood in zip(toronto_postals['Latitude'], toronto_postals['Longitude'], toronto_postals['Borough'],toronto_postals['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [15]:
CLIENT_ID = 'my foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'my secret' # your Foursquare Secret
VERSION = '20190605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: my foursquare ID
CLIENT_SECRET:my secret


In [16]:
toronto_postals.head()

,Postal Code,Borough,Neighbourhood,PostalCode,Latitude,Longitude
37,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",M4L,43.668999,-79.315572
43,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790


In [17]:
toronto_postals.reset_index(inplace=True)
latitude = str(toronto_postals.loc[0,'Latitude'])
longitude = str(toronto_postals.loc[0,'Longitude'])
search_query = "Indian"
radius = 500
LIMIT = 30

latitude
longitude

toronto_postals.iloc[0]

index                      37
Postal Code               M4E
Borough          East Toronto
Neighbourhood     The Beaches
PostalCode                M4E
Latitude              43.6764
Longitude             -79.293
Name: 0, dtype: object